### (magenta 패키지 설치없이) 4마디의 드럼 샘플 생성하기 try01(23.07.16)

- Magenta의 note-seq 패키지를 이용하여 MIDI (.mid, .midi) MusicXML (.xml, .mxl) 파일을 TFRecord 포맷의 NoteSequences으로 변환할 수 있음

- TFRecord 포맷은 바이너리 레코드를 저장하는 단순한 포맷입니다. TFRecord 포맷은 더 적은 공간으로 동일한 데이터를 저장할 수 있고, TPU와 같은 병렬 I/O 연산을 사용할 수 있습니다.

- 저는 magenta 패키지를 사용하지 않기 위해 mido를 이용했습니다.

In [1]:
import pandas as pd

In [2]:
info = pd.read_csv('/home/crb123/temp/groove-v1.0.0-midionly/groove/info.csv')

In [3]:
info

,drummer,session,id,style,bpm,beat_type,time_signature,midi_filename,audio_filename,duration,split
0,drummer1,drummer1/eval_session,drummer1/eval_session/1,funk/groove1,138,beat,4-4,drummer1/eval_session/1_funk-groove1_138_beat_...,drummer1/eval_session/1_funk-groove1_138_beat_...,27.872308,test
1,drummer1,drummer1/eval_session,drummer1/eval_session/10,soul/groove10,102,beat,4-4,drummer1/eval_session/10_soul-groove10_102_bea...,drummer1/eval_session/10_soul-groove10_102_bea...,37.691158,test
2,drummer1,drummer1/eval_session,drummer1/eval_session/2,funk/groove2,105,beat,4-4,drummer1/eval_session/2_funk-groove2_105_beat_...,drummer1/eval_session/2_funk-groove2_105_beat_...,36.351218,test
3,drummer1,drummer1/eval_session,drummer1/eval_session/3,soul/groove3,86,beat,4-4,drummer1/eval_session/3_soul-groove3_86_beat_4...,drummer1/eval_session/3_soul-groove3_86_beat_4...,44.716543,test
4,drummer1,drummer1/eval_session,drummer1/eval_session/4,soul/groove4,80,beat,4-4,drummer1/eval_session/4_soul-groove4_80_beat_4...,drummer1/eval_session/4_soul-groove4_80_beat_4...,47.987500,test
...,...,...,...,...,...,...,...,...,...,...,...
1145,drummer2,drummer2/session2,drummer2/session2/11,rock,130,beat,4-4,drummer2/session2/11_rock_130_beat_4-4.mid,NaN,1.909613,train
1146,drummer2,drummer2/session2,drummer2/session2/12,rock,130,beat,4-4,drummer2/session2/12_rock_130_beat_4-4.mid,NaN,1.808652,train
1147,drummer2,drummer2/session2,drummer2/session2/13,rock,130,beat,4-4,drummer2/session2/13_rock_130_beat_4-4.mid,NaN,1.864421,train
1148,drummer2,drummer2/session2,drummer2/session2/14,rock,130,beat,4-4,drummer2/session2/14_rock_130_beat_4-4.mid,NaN,1.875960,train


### 문제점
- 모든 드러머의 각 session 을 전부 합쳐서 학습데이터로 넣어도 되는것인지,
- 각 session의 역할과, 그 안에 있기도 한 eval의 역할이 무엇인지 파악하지 못함

In [ ]:
!pip install mido

In [14]:
import mido
import os
import tensorflow as tf

def midi_to_tfrecord(midi_dir, tfrecord_file):
    # MIDI 파일 목록 가져오기
    midi_files = [os.path.join(midi_dir, f) for f in os.listdir(midi_dir) if f.endswith('.mid')]

    # TFRecord 파일 작성기 생성
    with tf.io.TFRecordWriter(tfrecord_file) as writer:
        for midi_file in midi_files:
            # MIDI 파일 읽기
            mid = mido.MidiFile(midi_file)

            # 각 트랙에 대한 노트 이벤트 추출
            for track in mid.tracks:
                for msg in track:
                    if msg.type == 'note_on':
                        # 노트 이벤트를 TFRecord로 변환
                        feature = {
                            'note': tf.train.Feature(int64_list=tf.train.Int64List(value=[msg.note])),
                            'velocity': tf.train.Feature(int64_list=tf.train.Int64List(value=[msg.velocity])),
                            'time': tf.train.Feature(int64_list=tf.train.Int64List(value=[msg.time]))
                        }
                        example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
                        writer.write(example_proto.SerializeToString())


In [15]:
midi_to_tfrecord('/home/crb123/temp/groove-v1.0.0-midionly/groove/drummer1/session1/', 'notes.tfrecord')

In [2]:
import mido

midi_file = mido.MidiFile('/home/crb123/temp/groove-v1.0.0-midionly/groove/drummer1/session1/100_neworleans-secondline_94_beat_4-4.mid')


In [3]:
notes = []
for track in midi_file.tracks:
    for msg in track:
        if msg.type == 'note_on':
            # note_on 메시지에서 노트 번호와 벨로시티를 추출
            note = msg.note
            velocity = msg.velocity
            notes.append((note, velocity))


In [4]:
import tensorflow as tf

with tf.io.TFRecordWriter('notes.tfrecord') as writer:
    for note, velocity in notes:
        feature = {
            'note': tf.train.Feature(int64_list=tf.train.Int64List(value=[note])),
            'velocity': tf.train.Feature(int64_list=tf.train.Int64List(value=[velocity])),
        }
        example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
        writer.write(example_proto.SerializeToString())


In [6]:
import tensorflow as tf

# TFRecord 파일 로드
raw_dataset = tf.data.TFRecordDataset('notes.tfrecord')

# 특성 설명
feature_description = {
    'note': tf.io.FixedLenFeature([], tf.int64),
    'velocity': tf.io.FixedLenFeature([], tf.int64),
}

# TFRecord 파싱 함수
def _parse_function(example_proto):
    return tf.io.parse_single_example(example_proto, feature_description)

# TFRecord 파싱
parsed_dataset = raw_dataset.map(_parse_function)

# 파싱된 데이터 출력
for parsed_record in parsed_dataset.take(10):
    print(parsed_record)


{'note': <tf.Tensor: shape=(), dtype=int64, numpy=38>, 'velocity': <tf.Tensor: shape=(), dtype=int64, numpy=92>}
{'note': <tf.Tensor: shape=(), dtype=int64, numpy=38>, 'velocity': <tf.Tensor: shape=(), dtype=int64, numpy=0>}
{'note': <tf.Tensor: shape=(), dtype=int64, numpy=44>, 'velocity': <tf.Tensor: shape=(), dtype=int64, numpy=67>}
{'note': <tf.Tensor: shape=(), dtype=int64, numpy=44>, 'velocity': <tf.Tensor: shape=(), dtype=int64, numpy=0>}
{'note': <tf.Tensor: shape=(), dtype=int64, numpy=38>, 'velocity': <tf.Tensor: shape=(), dtype=int64, numpy=35>}
{'note': <tf.Tensor: shape=(), dtype=int64, numpy=38>, 'velocity': <tf.Tensor: shape=(), dtype=int64, numpy=0>}
{'note': <tf.Tensor: shape=(), dtype=int64, numpy=44>, 'velocity': <tf.Tensor: shape=(), dtype=int64, numpy=17>}
{'note': <tf.Tensor: shape=(), dtype=int64, numpy=38>, 'velocity': <tf.Tensor: shape=(), dtype=int64, numpy=45>}
{'note': <tf.Tensor: shape=(), dtype=int64, numpy=44>, 'velocity': <tf.Tensor: shape=(), dtype=int6

2023-07-16 21:28:49.869210: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-16 21:28:50.825931: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22665 MB memory:  -> device: 0, name: NVIDIA TITAN RTX, pci bus id: 0000:65:00.0, compute capability: 7.5
2023-07-16 21:28:51.224781: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [9]:
import tensorflow as tf
from tensorflow.keras import layers

class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

class Encoder(layers.Layer):
    def __init__(self, latent_dim=32, intermediate_dim=64, name="encoder", **kwargs):
        super(Encoder, self).__init__(name=name, **kwargs)
        self.lstm_layer = layers.LSTM(intermediate_dim, return_sequences=False)
        self.dense_mean = layers.Dense(latent_dim)
        self.dense_log_var = layers.Dense(latent_dim)
        self.sampling = Sampling()

    def call(self, inputs):
        x = self.lstm_layer(inputs)
        z_mean = self.dense_mean(x)
        z_log_var = self.dense_log_var(x)
        z = self.sampling((z_mean, z_log_var))
        return z_mean, z_log_var, z

class Decoder(layers.Layer):
    def __init__(self, original_dim, intermediate_dim=64, name="decoder", **kwargs):
        super(Decoder, self).__init__(name=name, **kwargs)
        self.lstm_layer = layers.LSTM(intermediate_dim, return_sequences=True)
        self.dense_output = layers.TimeDistributed(layers.Dense(original_dim))

    def call(self, inputs):
        x = self.lstm_layer(inputs)
        return self.dense_output(x)

class VariationalAutoEncoder(tf.keras.Model):
    def __init__(self, original_dim, intermediate_dim=64, latent_dim=32, name="autoencoder", **kwargs):
        super(VariationalAutoEncoder, self).__init__(name=name, **kwargs)
        self.original_dim = original_dim
        self.encoder = Encoder(latent_dim=latent_dim, intermediate_dim=intermediate_dim)
        self.decoder = Decoder(original_dim, intermediate_dim=intermediate_dim)

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        z = tf.expand_dims(z, 1)
        z = tf.repeat(z, repeats=inputs.shape[1], axis=1)
        reconstructed = self.decoder(z)
        kl_loss = -0.5 * tf.reduce_mean(z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
        self.add_loss(kl_loss)
        return reconstructed



In [27]:
import tensorflow as tf

# TFRecord 파일 로드
raw_dataset = tf.data.TFRecordDataset('notes.tfrecord')

# 특성 설명
feature_description = {
    'note': tf.io.FixedLenFeature([], tf.int64),
    'velocity': tf.io.FixedLenFeature([], tf.int64),
    'time': tf.io.FixedLenFeature([], tf.int64),  # 'time' 특성의 데이터 타입을 tf.int64로 변경
}

# TFRecord 파싱 함수
def _parse_function(example_proto):
    return tf.io.parse_single_example(example_proto, feature_description)

# TFRecord 파싱
parsed_dataset = raw_dataset.map(_parse_function)

# 파싱된 데이터를 numpy 배열로 변환
notes = []
for parsed_record in parsed_dataset:
    note = parsed_record['note'].numpy()
    velocity = parsed_record['velocity'].numpy()
    time = parsed_record['time'].numpy()  # 'time' 특성 추출
    notes.append([note, velocity, time])  # 'time' 특성 추가

# numpy 배열을 tensor로 변환
notes_tensor = tf.convert_to_tensor(notes, dtype=tf.float32)

# tensor의 요소 수가 sequence_length * 3의 배수가 되도록 조정
total_elements = notes_tensor.shape[0] * notes_tensor.shape[1]
multiple_of_sequence_length = total_elements // (sequence_length * 3) * (sequence_length * 3)
notes_tensor = notes_tensor[:multiple_of_sequence_length // 3]

# tensor를 시퀀스 형태로 변환
notes_sequence = tf.reshape(notes_tensor, (-1, sequence_length, 3))  # 3차원으로 변환

# VAE 모델 생성
vae = VariationalAutoEncoder(original_dim=3, intermediate_dim=64, latent_dim=32)  # original_dim을 3으로 변경

# 모델 훈련
vae.compile(optimizer='adam', loss='mse')
vae.fit(notes_sequence, notes_sequence, epochs=300)


Epoch 1/300
172/172 [==============================] - 3s 9ms/step - loss: 1422.9377
Epoch 2/300
172/172 [==============================] - 2s 9ms/step - loss: 1064.9060
Epoch 3/300
172/172 [==============================] - 2s 9ms/step - loss: 955.5722
Epoch 4/300
172/172 [==============================] - 2s 9ms/step - loss: 918.9689
Epoch 5/300
172/172 [==============================] - 2s 9ms/step - loss: 904.7917
Epoch 6/300
172/172 [==============================] - 2s 9ms/step - loss: 884.2988
Epoch 7/300
172/172 [==============================] - 2s 9ms/step - loss: 871.0427
Epoch 8/300
172/172 [==============================] - 2s 9ms/step - loss: 861.8123
Epoch 9/300
172/172 [==============================] - 2s 9ms/step - loss: 853.8077
Epoch 10/300
172/172 [==============================] - 2s 9ms/step - loss: 846.2877
Epoch 11/300
172/172 [==============================] - 2s 9ms/step - loss: 837.3560
Epoch 12/300
172/172 [==============================] - 2s 9ms/step - lo

In [28]:
import numpy as np

# 잠재 공간의 차원
latent_dim = 32

# 샘플링할 패턴의 수
num_samples = 10

# 잠재 공간에서 랜덤한 벡터 샘플링
random_latent_vectors = np.random.normal(size=(num_samples, latent_dim))

# 랜덤한 벡터를 디코더에 통과시켜 새로운 패턴 생성
random_latent_vectors = tf.convert_to_tensor(random_latent_vectors, dtype=tf.float32)
random_latent_vectors = tf.expand_dims(random_latent_vectors, 1)
random_latent_vectors = tf.repeat(random_latent_vectors, repeats=4, axis=1)  # 4마디
generated_patterns = vae.decoder(random_latent_vectors)

# 생성된 패턴 출력
print(generated_patterns)



tf.Tensor(
[[[ 5.36208057e+00 -3.87118292e+00  7.12517853e+01]
  [ 8.62187576e+00 -2.41719608e+01 -8.98176670e+00]
  [ 7.24905872e+00 -2.95041676e+01 -1.68507843e+01]
  [ 8.48229790e+00 -1.38150711e+01 -3.11280861e+01]]

 [[ 6.36191559e+00 -4.78437376e+00 -1.46785326e+01]
  [ 7.38080406e+00 -1.10686321e+01 -2.02827721e+01]
  [ 5.59035587e+00 -1.46230011e+01 -9.77155209e+00]
  [ 4.34164238e+00 -1.67527809e+01  1.06379213e+01]]

 [[ 1.32090168e+01  2.05360107e+01 -4.31128025e+00]
  [ 1.72518177e+01  1.80538368e+01  1.88336525e+01]
  [ 1.61558170e+01  6.17567205e+00  5.12949371e+01]
  [ 1.53624802e+01  3.52859640e+00  5.84367027e+01]]

 [[ 3.48096132e-01  6.04160538e+01  2.09785690e+01]
  [-4.14107561e+00  4.57705116e+01  3.92124634e+01]
  [-4.09208727e+00  7.93400726e+01  2.33230019e+01]
  [-1.62261391e+00  6.99820099e+01 -2.89738579e+01]]

 [[ 2.05955009e+01  2.87031517e+01  2.09578293e+02]
  [ 3.30621338e+01  2.49109993e+01  1.16910263e+02]
  [ 2.81986904e+01  2.26782990e+01  2.5637530

In [36]:
import mido

def patterns_to_midi(generated_patterns, output_file):
    mid = mido.MidiFile()
    track = mido.MidiTrack()
    mid.tracks.append(track)

    for pattern in generated_patterns[0]:
        note_number, velocity, time = pattern
        # MIDI 이벤트 생성
        note_on = mido.Message('note_on', note=note_number, velocity=velocity, time=time)
        note_off = mido.Message('note_off', note=note_number, velocity=velocity, time=time)
        track.append(note_on)
        track.append(note_off)

    mid.save(output_file)

In [31]:
import mido
import numpy as np

# 생성된 패턴을 MIDI 노트로 변환
notes = generated_patterns.numpy().astype(int)

# 새 MIDI 파일 생성
mid = mido.MidiFile()

# 새 트랙 생성
track = mido.MidiTrack()
mid.tracks.append(track)

# 변환된 MIDI 노트를 사용하여 MIDI 이벤트 생성
for note in notes[0]:
    note_number, velocity, time = note  # 'time' 값 추가
    # 'note', 'velocity', 'time' 값을 적절한 범위로 스케일링
    note_number = np.clip(note_number, 0, 127)
    velocity = np.clip(velocity, 0, 127)
    time = np.clip(time, 0, None)  # 'time' 값이 0 이상이 되도록
    print(time)
    # 노트 온 이벤트 생성
    note_on = mido.Message('note_on', note=note_number, velocity=velocity, time=time)  # 'time' 값 사용
    track.append(note_on)
    # 노트 오프 이벤트 생성
    note_off = mido.Message('note_off', note=note_number, velocity=velocity, time=time + 480)  # 'time' 값 사용
    track.append(note_off)

# MIDI 파일 저장
mid.save('generated_pattern.mid')


71
0
0
0


In [32]:
notes

array([[[  5,  -3,  71],
        [  8, -24,  -8],
        [  7, -29, -16],
        [  8, -13, -31]],

       [[  6,  -4, -14],
        [  7, -11, -20],
        [  5, -14,  -9],
        [  4, -16,  10]],

       [[ 13,  20,  -4],
        [ 17,  18,  18],
        [ 16,   6,  51],
        [ 15,   3,  58]],

       [[  0,  60,  20],
        [ -4,  45,  39],
        [ -4,  79,  23],
        [ -1,  69, -28]],

       [[ 20,  28, 209],
        [ 33,  24, 116],
        [ 28,  22, 256],
        [ 36,  18,  84]],

       [[  2,  -1,   2],
        [  1,  -2,   2],
        [  1,  -3,   3],
        [  0,  -3,   3]],

       [[  3,   0,   3],
        [  3,   0,   2],
        [  3,  -1,   1],
        [  3,  -1,   1]],

       [[  4,   0,   1],
        [  4,   1,  -1],
        [  4,   1,  -3],
        [  4,   1,  -5]],

       [[  1,   1,  42],
        [  2,  -2,  52],
        [  3,  -7,  63],
        [  1, -13,  99]],

       [[ -1,  -7, -11],
        [ -5, -11, -22],
        [ -5, -19, -19],
       

### 계획: Magenta 라이브러리를 사용하지 않고, 아래의 소스코드를  응용하여 tensorflow를 활용해 코드를 작성해보려고 했음 (미해결)

#### 소스코드 확인

In [ ]:
# 어느 부분을 어떻게 활용하는지 어려워 아래 블로그 참고하였습니다.
# https://medium.com/@seyong.dev/music-vae%EB%A1%9C-%EB%93%9C%EB%9F%BC-%EC%83%98%ED%94%8C-%EC%B6%94%EC%B6%9C%ED%95%98%EA%B8%B0-2e8d9f2dcecb

# 4마디 드럼 샘플을 생성하기 위해서 configs.py의 ‘cat-drums_4bar_big’하이퍼파라미터들 사용 
CONFIG_MAP['cat-drums_4bar_big'] = Config(
    model=MusicVAE(lstm_models.BidirectionalLstmEncoder(),
                   lstm_models.CategoricalLstmDecoder()),
    hparams=merge_hparams(
        lstm_models.get_default_hparams(),
        HParams(
            batch_size=512,
            max_seq_len=64,  # 4 bars w/ 16 steps per bar
            z_size=512,
            enc_rnn_size=[2048],
            dec_rnn_size=[2048, 2048, 2048],
            free_bits=48,
            max_beta=0.2,
            sampling_schedule='inverse_sigmoid',
            sampling_rate=1000,
        )),
    note_sequence_augmenter=None,
    data_converter=data.DrumsConverter(
        max_bars=100,  # Truncate long drum sequences before slicing.
        slice_bars=4,
        steps_per_quarter=4,
        roll_input=True),
    train_examples_path=None,
    eval_examples_path=None,
)


# 인코더로는 BidirectionalLstmEncoder 클래스를 사용하고 디코더로는 CategoricalLstmDecoder 클래스를 사용합니다. 
# 인코더의 경우는 2048개의 은닉 노드를 가진 양방향 LSTM 레이어 1개가 사용되었고, 
# 디코더의 경우에는 2048개의 은닉 노드를 가진 Categorical LSTM 레이어 3개가 사용되었습니다.
# lstm_models.py 의 BidirectionalLstmEncoder 클래스의 encode 함수
def encode(self, sequence, sequence_length):
    cells_fw, cells_bw = self._cells

    _, states_fw, states_bw = contrib_rnn.stack_bidirectional_dynamic_rnn(
        cells_fw,
        cells_bw,
        sequence,
        sequence_length=sequence_length,
        time_major=False,
        dtype=tf.float32,
        scope=self._name_or_scope)
    # Note we access the outputs (h) from the states since the backward
    # ouputs are reversed to the input order in the returned outputs.
    last_h_fw = states_fw[-1][-1].h
    last_h_bw = states_bw[-1][-1].h

    return tf.concat([last_h_fw, last_h_bw], 1)

# CategoricalLstmDecoder 클래스의 _sample 함수
# tfp.distributions.OneHotCategorical 메소드는 
# categorical distribution을 사용하여 샘플링을 수행
def _sample(self, rnn_output, temperature=1.0):
    sampler = tfp.distributions.OneHotCategorical(
        logits=rnn_output / temperature, dtype=tf.float32)
    return sampler.sample()


# preprocess_tfrecord.py 의 설정 중 드럼 시퀀스만 추출
flags.DEFINE_bool(
    'is_drum', True,
    'If None, filtering will consider drums and non-drums. If True, only drums '
    'will be considered. If False, only non-drums will be considered.')
flags.DEFINE_bool(
    'drums_only', True,
    'If True, NoteSequences with non-drum instruments will be skipped.')

# 훈련된 체크포인트를 이용해서 새로운 MIDI 데이터를 생성 
#  ‘Sample’ 방식과 ‘Interpolate’ 방식 중 선택
elif FLAGS.mode == 'sample':
    logging.info('Sampling...')
    results = model.sample(
        n=FLAGS.num_outputs,
        length=config.hparams.max_seq_len,
        temperature=FLAGS.temperature)

# tf.Graph()를 통해 그래프 생성
with tf.Graph().as_default():
    model = self._config.model
    model.build(
        self._config.hparams,
        self._config.data_converter.output_depth,
        is_training=False)
    
# batch_size와 z_size 크기의 플레이스홀더를 정의
if self._config.hparams.z_size:
    self._z_input = tf.placeholder(
        tf.float32, shape=[batch_size, self._config.hparams.z_size])
else:
    self._z_input = None

# 디코더 모델의 sample 함수에 batch_size, _max_length, _z_input, _c_input, _temperature 등의 
# 하이퍼파라미터를 입력값으로 넣어 _outputs를 출력
# _outputs는 이후의 sess.run() 함수에서 플레이스홀더 역할
self._outputs, self._decoder_results = model.sample(
    batch_size,
    max_length=self._max_length,
    z=self._z_input,
    c_input=self._c_input,
    temperature=self._temperature,
    **sample_kwargs)

# 아래의 코드로 체크포인트의 모델 가중치를 로드
self._sess = tf.Session(target = session_target)
saver = tf.train.Saver(var_map)
if (os.path.exists(checkpoint_paath) and
    tarfile.is_tarfile(checkpoint_path)):
    tf.logging.info('Unbundling checkpoint.')
    with tempfile.TemporaryDirectory() as temp_dir:
        tar = tarfile.open(checkpoint_path)
        tar.extractall(temp_dir)
        for name in tar.gennames():
            if name.endswith('.index'):
                checkpoint_path = os.path.join(temp_dir, name[0:-6])
                break
            saver.restore(self._sess, checkpoint_path)
else:
    saver.restore(self._sess, checkpoint_path)

# _c_input은 시퀀스를 제어(control)하는 입력값
outputs = []
for _ in range(int(np.ceil(n / batch_size))):
    if self._z_input is not None and not same_z:
    feed_dict[self._z_input] = (
        np.random.randn(batch_size, z_size).astype(np.float32))
    outputs.append(self._sess.run(self._outputs, feed_dict))
samples = np.vstack(outputs)[:n]
if self._c_input is not None:
    return self._config.data_converter.from_tensors(
        samples, np.tile(np.expand_dims(c_input, 0), [batch_size, 1, 1]))
else:
    return self._config.data_converter.from_tensors(samples)


# _c_input은 controls 변수의 위치에 해당하고, 멜로디, 베이스, 드럼 등 
# 여러가지 악기로 연주된 데이터에서 멜로디를 제어하는 역할 
# TrainedModel.sample()
def from_tensors(self, samples, controls=None):
    output_sequences = []
    dim_ranges = np.cumsum(self._split_output_depths)
    for i, s in enumerate(samples):
        mel_ns = self._melody_converter.from_tensors(
            [s[:, :dim_ranges[0]]],
            [controls[i]] if controls is not None else None)[0]
        bass_ns = self._melody_converter.from_tensors(
            [s[:, dim_ranges[0]:dim_ranges[1]]])[0]
        drums_ns = self._drums_converter.from_tensors(
            [s[:, dim_ranges[1]:]])[0]

    # LegacyEventListOneHotConverter.from_tensors()
    if self._chord_encoding and controls is not None:
        chords = [self._chord_encoding.decode_event(e)
                for e in np.argmax(controls[i][:, :-12], axis=-1)[:end_index]]
        chord_times = [step * seconds_per_step for step in event_list.steps]
        chords_lib.add_chords_to_sequence(sequence, chords, chord_times)
    if self._condition_on_key and controls is not None:
        keys = np.argmax(controls[i][:, -12:], axis=-1)[:end_index]
        key_times = [step * seconds_per_step for step in event_list.steps]
        chords_lib.add_keys_to_sequence(sequence, keys, key_times)
    output_sequences.append(sequence)
    return output_sequences


# NoteSequence 형식으로 반환된 샘플들은 아래 코드를 통해서 MIDI 파일로 변환(music_vae_generate.py)
for i, ns in enumerate(results):
    note_seq.sequence_proto_to_midi_file(ns, basename.replace('*', '%03d' % i))

# 결론
- magenta 패키지를 사용하지 않고 구현하는것이 어려웠습니다. (과제 완성률 10% 예상)
- 시간을 더 준다고 해낼 수 있을 지 의문이 들었습니다.
- 나름 시간을 들였으나 실력의 부족으로 완성을 하지 못했습니다. 스스로 많은 깨달음이 있었고 조금 더 노력해보아야 겠다는 다짐을 하게 되는 시간이 되었습니다.